# Undulator focusing and generation of a map file suitable for electron tracking


## _Import Radia and other packages_

In [ ]:
%matplotlib inline
import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
import time as tm
from math import *
from copy import *
from array import array

import radia as rad
import ipywidgets
from jupyter_rs_radia import radia_viewer

# import an illustration of this magnet
from IPython.display import Image
Image(filename=('./U46HybridUnd.png'))

## _Define function to build the undulator_

In [ ]:
def undulator(lp, mp, np, cp,  lm, mm, nm, cm,  gap, gapOffset, numPer):
    """
    create hybrid undulator magnet
    arguments:
      lp = [lpx, lpy, lpz] = dimensions of the iron poles / mm
      mp = magnetic properties of the iron poles (M-H curve)
      np = segmentation of the iron poles
      cp = [r,g,b] = color for the iron poles
      lm = [lmx, lmy, lmz] = dimensions of the magnet blocks / mm
      mm = magnetic properties of the magnet blocks (remanent magnetization)
      nm = segmentation of the magnet blocks
      cm = [r,g,b] = color for the magnet blocks
      gap = undulator gap / mm
      gapOffset = vertical offset / mm of the magnet blocks w/rt the poles
      numPer = number of full periods of the undulator magnetic field
    return: Radia representations of
      undulator group, poles, permanent magnets
    """
    zer = [0,0,0]

    # full magnet will be assembled into this Radia group
    grp = rad.ObjCnt([])

    # principal poles and magnet blocks in octant(+,+,–)
    # -- half pole
    y = lp[1]/4
    pole = rad.ObjFullMag([lp[0]/4,    y,    -lp[2]/2 - gap/2],
                          [lp[0]/2, lp[1]/2,  lp[2]          ], zer, np, grp, mp, cp)
    y += lp[1]/4
    # -- magnet and pole pairs
    mDir = -1
    for i in range(0, numPer):
        initM = [0, mDir, 0]; mDir *= -1
        y += lm[1]/2
        magnet = rad.ObjFullMag([lm[0]/4,   y,   -lm[2]/2 - gap/2 - gapOffset],
                                [lm[0]/2, lm[1],  lm[2]                      ], initM, nm, grp, mm, cm)
        y += (lm[1] + lp[1])/2
        pole = rad.ObjFullMag([lp[0]/4,   y,   -lp[2]/2 - gap/2],
                              [lp[0]/2, lp[1],  lp[2]          ], zer, np, grp, mp, cp)
        y += lp[1]/2
    # -- end magnet block
    initM = [0, mDir, 0]
    y += lm[1]/4;
    magnet = rad.ObjFullMag([lm[0]/4,   y,     -lm[2]/2 - gap/2 - gapOffset],
                            [lm[0]/2, lm[1]/2,  lm[2]                      ], initM, nm, grp, mm, cm)

    # use mirror symmetry to define the full undulator
    rad.TrfZerPerp(grp, zer, [1,0,0]) # reflect in the (y,z) plane
    rad.TrfZerPara(grp, zer, [0,0,1]) # reflect in the (x,y) plane
    rad.TrfZerPerp(grp, zer, [0,1,0]) # reflect in the (z,x) plane
    
    return grp, pole, magnet

In [ ]:
# From RadiaToTrack.m
#hybrid
def hyb(p,per,gap,nper,gapoffset, lp,np,matp,cp, lm,nm,matm,cm, lms,nms,matms,cms):
    zer = [0,0,0]
    inn = -1
    grp = rad.ObjCnt([])

    yy = lp[1]/4
    rad.ObjFullMag(p + [lp[0]/4, yy, -lp[2]/2 - gap/2], [lp[0]/2, lp[1]/2, lp[2]], zer, np, grp, matp, cp)

    yys = yy + (lms[1] - lp[1])/4
    rad.ObjFullMag(p + [lp[0]/2 + lms[0]/2, yys, -lms[2]/2 - gap/2 - gapoffset], [lms[0], lms[1]/2, lms[2]],[inn, 0, 0], nms, grp, matms, cms)

    yy += lp[1]/4
    for i in range(1,nper+1):
        initm = [0, (i + 1)%2 - i%2, 0]
        yy += lm[1]/2
        rad.ObjFullMag(p + [lm[0]/4, yy, -lm[2]/2 - gap/2 - gapoffset], [lm[0]/2, lm[1], lm[2]], initm, nm, grp, matm, cm)
        yy += (lm[1] + lp[1])/2;
        rad.ObjFullMag(p + [lp[0]/4, yy, -lp[2]/2 - gap/2], [lp[0]/2, lp[1], lp[2]], zer, np, grp, matp, cp)
        inn *= -1
        rad.ObjFullMag(p + [lp[0]/2 + lms[0]/2, yy, -lms[2]/2 - gap/2 - gapoffset], [lms[0], lms[1], lms[2]], [inn, 0, 0], nms, grp, matms, cms)
  
        yy += lp[1]/2
    initm = [0, nper%2 - (nper + 1)%2, 0]
    yy += lm[1]/4
    rad.ObjFullMag(p + [lm[0]/4, yy, -lm[2]/2 - gap/2 - gapoffset], [lm[0]/2, lm[1]/2, lm[2]], initm, nm, grp, matm, cm)

    rad.TrfZerPerp(grp, p, [1,0,0]) # reflect in the (y,z) plane
    rad.TrfZerPara(grp, p, [0,0,1]) # reflect in the (x,y) plane
    rad.TrfZerPerp(grp, p, [0,1,0]) # reflect in the (z,x) plane
    
    return grp
    
# Pure Permanent Magnet
def undparts(po, wv, wh, nnp, per, br, si, axe=0.):
    g = rad.ObjCnt([])
    p = po - [0, nnp*per/2, 0]
    for i in range(0,4*nnp + 1):
        if i == 0 or i == 4*nnp: s = 0.5
        else: s = 1.
        if i%2 == 0: w = wv
        else: w = wh
        t = -(i - 1)*np.pi/2*si
        m = np.array([np.sin(axe)*np.sin(t), np.cos(t), np.cos(axe)*np.sin(t)])*br*s
        ma = rad.ObjRecMag(p, w, m)
        rad.ObjAddToCnt(g, [ma])
        p = p + [0, per/4, 0]
    rad.ObjDrwAtr(g, [0, 0, 1])
    return g

# "Apple II" type undulator
def apple2G(pos, per, gap, gapx, plr, pll, pur, pul, lx, lz, airgap, br, nper):
    wv = [lx/2, per/4 - airgap, lz]
    wh = wv
    px = lx/4 + gapx/2
    pz = gap/2 + lz/2
    
    g1 = undparts(pos + [px, pur, pz], wv, wh, nper, per, br, si=1)
    g2 = undparts(pos + [-px, pul, pz], wv, wh, nper, per, br, si=1)
    g3 = undparts(pos + [px, plr, -pz], wv, wh, nper, per, -br, si=-1)
    g4 = undparts(pos + [-px, pll, -pz], wv, wh, nper, per, -br, si=-1)

    g = rad.ObjCnt([g1, g2, g3, g4])
    return g

def apple2(pos, per, gap, gapx, phase, lx, lz, airgap, br, nper):
    wv = [lx/2, per/4 - airgap, lz]
    wh = wv
    px = lx/4 + gapx/2
    pz = gap/2 + lz/2
    
    g1 = undparts(pos + [px, phase/2, pz], wv, wh, nper, per, br, 1)
    g2 = undparts(pos + [-px, -phase/2, pz], wv, wh, nper, per, br, 1)
    g3 = undparts(pos + [px, -phase/2, -pz], wv, wh, nper, per, -br, -1)
    g4 = undparts(pos + [-px, phase/2, -pz], wv, wh, nper, per, -br, -1)
    g = rad.ObjCnt([g1, g2, g3, g4])
    return g

#Spring8
def sp8(pos, per, gap, gapx, phase, lxc, lzc, colc, lxs, lzs, cols, airgap, br, nper):
    wc = [lxc, per/4 - airgap, lzc]
    px = 0
    pz = gap/2 + lzc/2
    g1 = undparts(pos + [px, -phase/2, pz], wc, wc, nper, per, br, 1)
    rad.ObjDrwAtr(g1, colc)
    g2 = undparts(pos + [px, -phase/2, -pz], wc, wc, nper, per, -br, -1)
    rad.ObjDrwAtr(g2, colc)
    wc = [lxs, per/4 - airgap, lzs]
    px = lxc/2 + gapx + lxs/2
    pz = gap/2 + lzs/2
    
    g3 = undparts(pos + [px, phase/2, pz], wc, wc, nper, per, br, 1)
    rad.ObjDrwAtr(g3, cols)
    g4 = undparts(pos + [px, phase/2, -pz], wc, wc, nper, per, br, -1)
    rad.ObjDrwAtr(g4, cols)
    g5 = undparts(pos + [-px, phase/2, pz], wc, wc, nper, per, -br, 1)
    rad.ObjDrwAtr(g5, cols)
    g6 = undparts(pos + [-px, phase/2, -pz], wc, wc, nper, per, -br, -1)
    rad.ObjDrwAtr(g6, cols)
    
    g = rad.ObjCnt([g1, g2, g3, g4, g5, g6])
    return g

In [ ]:
# From RADIA_APPLE_II_Demo.py
def MagnetBlock(_pc, _wc, _cx, _cz, _type, _ndiv, _m):

    u = rad.ObjCnt([])
    
    wwc = _wc
    if(_type!=0):
        wwc = copy(_wc)
        wwc[0] -= 2*_cx
        
    b1 = rad.ObjRecMag(_pc, wwc, _m)
    rad.ObjAddToCnt(u, [b1])
    #ndiv2 = [1,_ndiv[1],_ndiv[2]]

    if((_cx>0.01) and (_cz>0.01)):
        if(_type==1):
            ppc = [_pc[0]-_wc[0]/2+_cx/2,_pc[1],_pc[2]-_cz/2]
            wwc = [_cx,_wc[1],_wc[2]-_cz]
            b2 = rad.ObjRecMag(ppc, wwc, _m)

            ppc = [_pc[0]+_wc[0]/2-_cx/2,_pc[1],_pc[2]+_cz/2]
            wwc = [_cx,_wc[1],_wc[2]-_cz]
            b3 = rad.ObjRecMag(ppc, wwc, _m)
            rad.ObjAddToCnt(u, [b2,b3])

        elif(_type==2):
            ppc = [_pc[0]-_wc[0]/2+_cx/2,_pc[1],_pc[2]+_cz/2]
            wwc = [_cx,_wc[1],_wc[2]-_cz]
            b2 = rad.ObjRecMag(ppc, wwc, _m)

            ppc = [_pc[0]+_wc[0]/2-_cx/2,_pc[1],_pc[2]-_cz/2]
            wwc = [_cx,_wc[1],_wc[2]-_cz]
            b3 = rad.ObjRecMag(ppc, wwc, _m)
            rad.ObjAddToCnt(u, [b2,b3])

        elif(_type==3):
            ppc = [_pc[0]-_wc[0]/2+_cx/2,_pc[1],_pc[2]]
            wwc = [_cx,_wc[1],_wc[2]-2*_cz]
            b2 = rad.ObjRecMag(ppc, wwc, _m)

            ppc = [_pc[0]+_wc[0]/2-_cx/2,_pc[1],_pc[2]]
            wwc = [_cx,_wc[1],_wc[2]-2*_cz]
            b3 = rad.ObjRecMag(ppc, wwc, _m)
            rad.ObjAddToCnt(u, [b2,b3])

    rad.ObjDivMag(u, _ndiv, 'Frame->LabTot')
    return u

#*************Magnet Array
def MagnetArray(_per, _nper, _po, _w, _si, _type, _cx, _cz, _br, _mu, _ndiv, _bs1, _s1, _bs2, _s2, _bs3, _s3, _bs2dz=0, _qp_ind_mag=None, _qp_dz=0):

    u = rad.ObjCnt([])

    Le = _bs1+_s1+_bs2+_s2+_bs3+_s3
    Lc = (_nper+0.25)*_per
    p = [_po[0],_po[1]-(Lc/2+Le),_po[2]] #po-{0,(Lc/2+Le),0}

    nMagTot = 4*_nper+7
    iMagCen = int(nMagTot/2.) #0-based index of the central magnet
    #print('iMagCen =', iMagCen) #DEBUG

    QP_IsDef = False; QP_DispIsConst = True
    nQP_Disp = 0
    if(_qp_ind_mag is not None):
        if(isinstance(_qp_ind_mag, list) or isinstance(_qp_ind_mag, array)):
            nQP_Disp = len(_qp_ind_mag)
            if(nQP_Disp > 0): QP_IsDef = True
        if(isinstance(_qp_dz, list) or isinstance(_qp_dz, array)): QP_DispIsConst = False
        elif(_qp_dz==0): QP_IsDef = False

    for i in range(nMagTot):
        wc = copy(_w)
        
        if(i==0):
            p[1] += _bs1/2
            wc[1] = _bs1
        elif(i==1):
            p[1] += _bs1/2+_s1+_bs2/2
            wc[1] = _bs2
        elif(i==2):
            p[1] += _bs2/2+_s2+_bs3/2
            wc[1] = _bs3
        elif(i==3):
            p[1] += _bs3/2+_s3+_per/8
        elif((i>3) and (i<4*_nper+4)):
            p[1] += _per/4
        elif(i==4*_nper+4):
            p[1] += _per/8+_s3+_bs3/2
            wc[1] = _bs3
        elif(i==4*_nper+5):
            p[1] += _bs3/2+_s2+_bs2/2
            wc[1] = _bs2
        elif(i==4*_nper+6):
            p[1] += _bs2/2+_s1+_bs1/2
            wc[1] = _bs1

        pc = copy(p)

        if((i==1) or (i==4*_nper+5)):
            if(_si==1): pc[2] += _bs2dz
            else: pc[2] -= _bs2dz

        if(QP_IsDef):
            for iQP in range(nQP_Disp):
                if(i == _qp_ind_mag[iQP] + iMagCen):
                    qpdz = _qp_dz
                    if(not QP_DispIsConst): qpdz = _qp_dz[iQP]
                    pc[2] += qpdz
                    #print('Abs. Ind. of Mag. to be Displaced:', i) #DEBUG
                    break

        t = -i*pi/2*_si
        mcol = [0.0,cos(t),sin(t)]
        m = [mcol[0],mcol[1]*_br,mcol[2]*_br]

        ma = MagnetBlock(pc, wc, _cx, _cz, _type, _ndiv, m)

        mcol = [0.27, 0.9*abs(mcol[1]), 0.9*abs(mcol[2])]
        rad.ObjDrwAtr(ma, mcol, 0.0001)

        rad.ObjAddToCnt(u, [ma])

    mat = rad.MatLin(_mu, abs(_br))
    rad.MatApl(u, mat)

    return u 

#*************Undulator
def APPLE_II(_per, _nper, _gap, _gapx, _phase, _phase_type, _lx, _lz, _cx, _cz, _air, _br, _mu, _ndiv, _bs1, _s1, _bs2, _s2, _bs3, _s3, _bs2dz, _qp_ind_mag, _qp_dz, _use_sym=False):

    w = [_lx,_per/4-_air,_lz]
    px = _lx/2+_gapx/2;
    pz = _gap/2+_lz/2;

    p1 = 0; p2 = _phase; p3 = 0; p4 = _phase
    if(_phase_type < 0): p2 = -_phase

    #print('w =', w)
    g1 = MagnetArray(_per, _nper, _po=[px,p1,pz], _w=w, _si=1, _type=1, _cx=_cx, _cz=_cz, _br=_br, _mu=_mu, _ndiv=_ndiv,
                     _bs1=_bs1, _s1=_s1, _bs2=_bs2, _s2=_s2, _bs3=_bs3, _s3=_s3, _bs2dz=_bs2dz, _qp_ind_mag=_qp_ind_mag, _qp_dz=_qp_dz)

    g2 = MagnetArray(_per, _nper, _po=[-px,p2,pz], _w=w, _si=1, _type=2, _cx=_cx, _cz=_cz, _br=_br, _mu=_mu, _ndiv=_ndiv,
                     _bs1=_bs1, _s1=_s1, _bs2=_bs2, _s2=_s2, _bs3=_bs3, _s3=_s3, _bs2dz=_bs2dz, _qp_ind_mag=_qp_ind_mag, _qp_dz=_qp_dz)

    if(_use_sym):
        u = rad.ObjCnt([g1,g2])
        trf = rad.TrfCmbL(rad.TrfRot([0,0,0],[0,1,0],pi), rad.TrfInv())
        rad.TrfMlt(u, trf, 2)
        return u, g1, g2, 0, 0

    g3 = MagnetArray(_per, _nper, _po=[-px,p3,-pz], _w=w, _si=-1, _type=1, _cx=_cx, _cz=_cz, _br=-_br, _mu=_mu, _ndiv=_ndiv,
                     _bs1=_bs1, _s1=_s1, _bs2=_bs2, _s2=_s2, _bs3=_bs3, _s3=_s3, _bs2dz=_bs2dz, _qp_ind_mag=_qp_ind_mag, _qp_dz=_qp_dz)

    g4 = MagnetArray(_per, _nper, _po=[px,p4,-pz], _w=w, _si=-1, _type=2, _cx=_cx, _cz=_cz, _br=-_br, _mu=_mu, _ndiv=_ndiv,
                     _bs1=_bs1, _s1=_s1, _bs2=_bs2, _s2=_s2, _bs3=_bs3, _s3=_s3, _bs2dz=_bs2dz, _qp_ind_mag=_qp_ind_mag, _qp_dz=_qp_dz)

    u = rad.ObjCnt([g1,g2,g3,g4])
    return u, g1, g2, g3, g4



## _Define functions for the magnetic materials and fields_

The first of the following functions returns a Radia representation of the magnetization curves _M_ (in Tesla) vs _H_ (in Amp/m) for both the pole tips and the magnets. The material for the poles is defined by calling the function `rad.MatSatIsoTab` with the data table `ma`, which contains the paired values of _H_ and _M_, both expressed in Tesla. (The conversion of _H_ in Amp/m to Tesla is accomplished simply by multiplying _H_ by _μ<sub>o</sub>_, i.e. `4*pi/10^7`.) The material for the permanent magnets is defined by calling `rad.MatStd`, with arguments _material type_ and _remanent magnetization_. We will later plot the resulting _M_(_H_) curves for verification.

The next function extracts magnetic field information from a given magnetic material defined by Radia. This allows us to plot the _M_(_H_) curves tocompare with our input data. And the third function calculates (for a _solved_ Radia magnet) the vertical field along the Y (longitudinal) axis.

In [ ]:
# magnetic constant (vacuum magnetic permeability)
mu0 = 4 * np.pi / 1e7

def materials(H, M,  smat, rm):
    """
    define magnetic materials for the undulator poles and magnets
    arguments:
      H    = list of magnetic field values / (Amp/m)
      M    = corresponding magnetization values / T
      smat = material type string
      rm   = remanent magnetization / T
    return: Radia representations of ...
      pole-tip material, magnet material
    """
    # -- magnetic property of poles
    #convH = 4. * np.pi / 1.0e7
    ma = [[mu0 * H[i], M[i]] for i in range(len(H))]
    mp = rad.MatSatIsoTab(ma)
    # -- permanent magnet material
    mm = rad.MatStd(smat, rm)

    return mp, mm


def getMagnMaterCompMvsH(meshH, ind, cmpnH,  cmpnM):
    """
    extract magnetization M_comp vs field strength H_comp from material data
    arguments:
      meshH = how to mesh the H axis: [hmin, hmax, nh]
      ind   = magnet material index
      cmpnH = component of H ('x', 'y', or 'z')
      cmpnM = component of M ('x', 'y', or 'z')
    return:
      given component of magnetization along the given mesh in H
    """
    hMin = meshH[0]; hMax = meshH[1]; nh = meshH[2]
    hStep = (hMax - hMin) / (nh - 1)
    m = [ 0 ] * nh
    sCmpnM = 'm' + cmpnM
    h = hMin
    H = [0,0,0]
    for i in range(nh):
        if(  cmpnH == 'x'): H[0] = h
        elif(cmpnH == 'y'): H[1] = h
        elif(cmpnH == 'z'): H[2] = h
        m[i] = rad.MatMvsH(ind, sCmpnM, H)
        h += hStep
    return m


def calcField(grp, per, numPer):
    """
    compute vertical magnetic field vs longitudinal position (Y)
    arguments:
      grp    = magnet (Radia group)
      per    = period / mm
      numPer = number of full magnetic periods
    return:
      Bz(y), meshY
    """
    xc = 0 # transverse X coordinate
    zc = 0 # transverse Z coordinate
    yMax = per * (numPer + 1) / 2.
    yMin = -yMax
    ny = 501
    yStep = (yMax - yMin) / ny
    y = np.arange(yMin, yMin + ny * yStep, yStep)
    
    points = [[xc, yi, zc] for yi in y]
    BzVsY = rad.Fld(grp, 'bz', points)

    return BzVsY, [yMin, yMax, ny]

## _Build and visualize an undulator_

###  _Vertical Field,  Pure Permanent Magnet_

In [ ]:
per = 35        #Undulator period in [mm]
gap = 10        #Magnetic Gap in [mm]
br = 1.2        #Remanent Field of the material in [T]
nper = 3        #Number of periods
lx = 30         #Horizontal width of the magnet blocks in [mm]
lz = 18         #Vertical height of the magnet blocks in [mm]
airgap = 0.5    #Airgap between the magnets in [mm]

#________________  Start Computations ___________________
wv = [lx, per/4 - airgap, lz]
wh = wv
p = np.array([0, 0, 0])
g = undparts(p + [0, 0, gap/2 + lz/2], wv, wh, nper, per, br, 1)
rad.TrfZerPara(g, p, [0, 0, 1])

# set up the radia viewer and display geometry
rvX = radia_viewer.RadiaViewer()
rvX.add_geometry('Vertical Field,  Pure Permanent Magnet', g)
rvX.display()

###  _"Hybrid" with side Magnets_
####  _U46 Hybrid Undulator_
First enter and plot the data describing the magnetization curve for the iron pole tips. Then set the magnet parameters and materials, and “build the magnet”.Then set the magnet parameters and materials, and “build the magnet”.

In [ ]:
# ~iron type Va Permendur
ironH = [  0.8,   1.5,     2.2,    3.6,    5.0,     6.8,     9.8,    18.0,
          28.0,   37.5,   42.0,   55.0,   71.5,    80.0,    85.0,    88.0,
          92.0,  100.0,  120.0,  150.0,  200.0,   300.0,   400.0,   600.0,
         800.0, 1000.0, 2000.0, 4000.0, 6000.0, 10000.0, 25000.0, 40000.0]
ironM = [0.000998995, 0.00199812, 0.00299724, 0.00499548, 0.00699372, 0.00999145, 0.0149877, 0.0299774,
         0.0499648,   0.0799529,  0.0999472,  0.199931,   0.49991,    0.799899,   0.999893, 1.09989,
         1.19988,     1.29987,    1.41985,    1.49981,    1.59975,    1.72962,    1.7995,   1.89925,
         1.96899,     1.99874,    2.09749,    2.19497,    2.24246,    2.27743,    2.28958,  2.28973]
# set parameters for this undulator
# -- general parameters
nper =  2    # number of full magnetic periods
per = 46       # period / mm
gap = 20       # gap height / mm
gapOffset = 1  # 

# -- parameters for the iron poles
lp =  [45,5,25] # dimensions / mm
nsp = [2,2,5]   # pole-tip segmentation (try varying this setting)
cp =  [1,0,1]   # assign color
ll = per/2 - lp[1]

# -- parameters for the magnet block
lm  = [65,ll,45] # dimensions / mm
nsm = [1,3,1]    # magnet-block segmentation (try varying this)
cm  = [0,1,1]    # assign color

# -- magnetic materials
# pole tips: ~iron type Va Permendur
# permanent magnets: NdFeB with 1.2 Tesla remanent magnetization
mp, mm = materials(ironH, ironM, 'NdFeB', 1.2)

# then build the undulator
und, pole, magnet = undulator(lp, mp, nsp, cp, lm, mm, nsm, cm, gap, gapOffset, nper)

# set up the radia viewer and display geometry
rv = radia_viewer.RadiaViewer()
rv.add_geometry('U46 Hybrid Undulator', und)
rv.display()

####  _SPEAR BL11 Wiggler_

In [ ]:
gap = 16
nper = 3
per = 175
gapoffset = 5
br = 1.25

lp = [50,42,90]
nsp = [2,2,10]
cp = [1,0,1]

lm = [126,per/2-lp[1],110]
nsm = [1,3,1]
cm = [0,1,1]

lms = [36,lp[1]-1,110]
nsms = [1,1,1]
cms = [0,1,0]

matp = rad.MatStd('Steel42')
matm = rad.MatStd('NdFeB', br)
matms = rad.MatStd('NdFeB', br)

p = np.array([0,0,0])
SPEAR = hyb(p,per,gap,nper,gapoffset, lp,nsp,matp,cp, lm,nsm,matm,cm, lms,nsms,matms,cms)

# set up the radia viewer and display geometry
rvX = radia_viewer.RadiaViewer()
rvX.add_geometry('SPEAR BLII Wiggler', SPEAR)
rvX.display()

### _"Apple II" Type Helical/Planar_

#### _ESRF  HU88 on ID8_

In [ ]:
per = 88                 # Undulator Period in [mm] 
lx = 25*2                 # Total Horizontal width of the magnet array in [mm]
lz = 12                 # Vertical height of the magnet blocks
gapx = 1                 # Horizontal gap between magnet arrays in [mm]
gap = 16                 # Magnetic Gap in[mm]
p1 = per*0.0                 # Antisym.  motion of the girders  : Rotate the Linear Pol.
p2 = per*0.5                 # Symmetric L motion of the girders : Linear\[Rule] Circular\[Rule] Linear
br = 1.2                 # Remanent field of the magnet blocks in [T]
nper = 6                 # Number of periods
airgap = 0                 # longitudinal airgap between magnets in [mm]

#Start Computations
p = np.array([0, 0, 0])
HU88 = apple2G(p, per, gap, gapx, -p1 + p2/2, -p2/2, -p2/2, p1 + p2/2, lx, lz, airgap, br, nper)

# set up the radia viewer and display geometry
rvX = radia_viewer.RadiaViewer()
rvX.add_geometry('ESRF  HU88 on ID8', HU88)
rvX.display()

#### _ESRF  HU38 on ID12_

In [ ]:
per = 38                 # Undulator Period in [mm] 
lx = 25*2                 # Total Horizontal width of the magnet array in [mm]
lz = 15                 # Vertical height of the magnet blocks
gapx = 1                 # Horizontal gap between magnet arrays in [mm]
gap = 16                 # Magnetic Gap in[mm]
p1 = per*0.0                 # Antisym.  motion of the girders  : Rotate the Linear Pol.
p2 = per*0.5                 # Symmetric L motion of the girders : Linear\[Rule] Circular\[Rule] Linear
br = 1.2                 # Remanent field of the magnet blocks in [T]
nper = 4                 # Number of periods
airgap = 0                 # longitudinal airgap between magnets in [mm]

#Start Computations
p = np.array([0, 0, 0])
HU38 = apple2G(p, per, gap, gapx, -p1 + p2/2, -p2/2, -p2/2, p1 + p2/2, lx, lz, airgap, br, nper)

# set up the radia viewer and display geometry
rvX = radia_viewer.RadiaViewer()
rvX.add_geometry('ESRF  HU38 on ID12', HU38)
rvX.display()

#### _APPLE-II type PPM Undulator_

In [ ]:
#Undulator Geometry Parameters (all lengths are in [mm])
per = 50. #Period
nper = 40 #38 #Number of Periods (should be EVEN here to make correct integer (M3+M5) modular structure!)

gapx = 1. #Horizontal Gap between Magnet Arrays
gap = 11.5 #Vertical Magnetic Gap
phase = 0. #0.2923*per #0.29*per #0.28*per #per/2. #per/4. #Longitudinal Shift between Magnet Arrays
phaseType = 1 #=1 means Parallel, -1 Anti-Parallel displacement of Magnet Arrays

lx = 34. #Horizontal Magnet Size
lz = 34. #Vertical Magnet Size
cx = 5. #Horizontal Notch size
cz = 5. #Vertical Notch size

air = 0.05 #Air Space between Magnets in Longitudinal Direction

#Terminations
bs3 = 7.25*per/49.2 #8.43/57.2*per (*7.89*per/105.2;*) (*7.25*per/49.2;*)  (*11.2*per/80.;*) #Magnet D1
bs2 = 6.11*per/49.2 #7.09/57.2*per #(*13.10*per/105.2;*) (*6.11*per/49.2;*) (*9.985*per/80.;*) #Magnet C
bs1 = 3.82*per/49.2 #6.58/57.2*per #(*13.56*per/105.2;*)(*3.82*per/49.2;*) (*7.9*per/80.;*) #Magnet D2
s1 = 1.*per/49.2 #1.41*per/57.2 #(*4.63*per/105.2;*)  (*1.*per/49.2;*)   (*2.4261*per/80.; *) #Outmost Gap G2
s2 = 4.5*per/49.2 #3.84*per/57.2 #(*2.47*per/105.2;*) (*4.5*per/49.2;*) (*5.7595*per/80.;*) #Inmost Gap G1
s3 = 1.*per/49.2 #0. #(*1.*per/49.2;*) (*1.;*)(*0.;*)
bs2dz = 0. #(*0.16961;*) #Vertical displacement of vertically-magnetised termination block

#Material Constants
br = 1.27 #Remanent Magnetization
mu = [0.05,0.15] #[0.06,0.17] #Suscettivita

#Subdivision Params
nDiv = [3,2,3] 

#Quasi-Periodic Perturbation
indsMagDispQP = [-77, -63, -49, -35, -21, -7, 7, 21, 35, 49, 63, 77] #None #Indexes of magnets counting from the central magnet of the structure, which has index 0.
vertMagDispQP = 0. 

#Start Computations
und = APPLE_II(_per=per, _nper=nper, _gap=gap, _gapx=gapx, _phase=phase, _phase_type=phaseType, _lx=lx, _lz=lz, _cx=cx, _cz=cz, _air=air,
                   _br=br, _mu=mu, _ndiv=nDiv, _bs1=bs1, _s1=s1, _bs2=bs2, _s2=s2, _bs3=bs3, _s3=s3, _bs2dz=bs2dz,
                   _qp_ind_mag=indsMagDispQP, _qp_dz=vertMagDispQP, _use_sym=True)[0]

# set up the radia viewer and display geometry
rvX = radia_viewer.RadiaViewer()
rvX.add_geometry('APPLE-II type PPM Undulator', und)
rvX.display()

### _"Spring8" Type Helical/Planar_

In [ ]:
per = 88                 #Undulator Period in [mm]
gapx = 1                 #Horizontal gap between magnet arrays in [mm]
gap = 16                 #Magnetic Gap in[mm]
phase = per/4            #Phase in [mm]
lxc = 14                 #Horizontal size of the magnets of the middle array
lzc = 12                 #Vertical size of the magnets of the middle array
colc = [1, 0, 0]         #RGB color of the middle array
lxs = 25                 #Horizontal size of the magnets of the side arrays
lzs = 16                 #Vertical size of the magnets of the side arrays
cols = [0, 1, 0]         #RGB color of the side arrays
br = 1.2                 #Remanent field of the magnet blocks in [T]
nper = 4                 #Number of periods
airgap = 0.1             #longitudinal airgap between magnets in [mm]

#________________  Start Computations ___________________
# rad.UtiDelAll()
p = np.array([0, 0, 0])
Spring8 = sp8(p, per, gap, gapx, phase, lxc, lzc, colc, lxs, lzs, cols, airgap, br, nper)

# set up the radia viewer and display geometry
rvX = radia_viewer.RadiaViewer()
rvX.add_geometry('"Spring8" Type Helical/Planar', Spring8)
rvX.display()

## _Solve for the undulator magnetization_

Here we ask Radia to solve for the undulator magnetization—_i.e._ for the magnetization within each sub-block that constitutes the undulator—and also report some statistics concerning that solution.

In [ ]:
# solve for the magnetization
prec = 0.0003  # precision for this computation
maxIter = 1000 # maximum allowed iterations
t0 = tm.time()
res = rad.Solve(und, prec, maxIter)
t1 = tm.time()

print("Solved for magnetization in {0:6f} s".format(t1 - t0))
print("Relaxation results")
print("  number of iterations: {0:5d}".format(int(res[3])))
print("  average stability of magnetization at last iteration: {0:.4e} T".format(res[0]))
print("  maximum absolute magnetization at last iteration: {0:.5f} T".format(res[1]))
print("  maximum H vector at last iteration: {0:.5f} T".format(res[2]))
print("Peak magnetic field: {0:.8f} T".format(rad.Fld(und, 'bz', [0,0,0])))

Now ask Radia to compute the magnetic field along the undulator’s longitudinal axis. And then plot the results.

In [ ]:
# compute magnetic field values
t2 = tm.time()
BzVsY, MeshY = calcField(und, per, nper)
t3 = tm.time()
print("Computed requested field vaues in {0:6f} s\n".format(t3 - t2))

# and plot
y = np.linspace(MeshY[0], MeshY[1], MeshY[2])
plt.figure(figsize=(7,4.3))
plt.plot(y, BzVsY, linewidth = 2.0)
plt.title(r"Vertical Magnetic Field, $B_z$, along undulator's $y$ axis")
plt.xlabel('y / mm')
plt.ylabel('$B_z$ / T')
plt.grid()
plt.show()

## _First Field Integral_

In [ ]:
# compute the first field integral at the point outside the undulator
ypoint = per*(nper+1)
Bz_int1st_point = rad.FldInt(und,'fin','ibz',[0,-1e5,0],[0,ypoint,0])
print("the 1st field integral outside the undulator is: ", Bz_int1st_point)

In [ ]:
# compute the first field integral along the longitudinal axis
yy = np.linspace(-per*(nper+1)/2.,ypoint, 501)
Bz_int1st = np.zeros_like(yy)
for i, yi in enumerate(yy):
    if i == 0:
        Bz_int1st[i] = rad.FldInt(und,'fin','ibz',[0,-1e5,0],[0,yi,0])
    else:
        Bz_int1st[i] = Bz_int1st[i-1] + rad.FldInt(und,'fin','ibz',[0,yy[i-1],0],[0,yi,0])
print("the 1st field integral outside the undulator is: ", Bz_int1st[-1])

In [ ]:
#plot the first field integral along the longitudinal axis
plt.figure(figsize=(7,4.3))
plt.plot(yy, Bz_int1st, linewidth = 2.0)
plt.title(r"1st Field integral along undulator's $y$ axis")
plt.xlabel('y / mm')
plt.ylabel('$B_z$ / T')
plt.grid()
plt.show()

## _Optimizing Parameters_

The fundamental undulator parameter K, is given by
$$K = \frac{e B_0 \lambda_u}{2\pi m_e c}.$$

In [ ]:
def undulatorK_simple(obj, per, pf_loc = [0,0,0] ,prec = 1e-5, maxIter = 10000, lprint = True):
    """
    compute undulator K value
    arguments:
      obj = undulator object
      per = undulator period / m
      pf_loc = peak field location [x, y, z]
      prec = precision goal for this computation
      maxIter = maximum allowed iterations
      lprint: whether or not to print results
    return:
      K = (e B_0 \lambda_u) / (2\pi m_e c)
    """
    res = rad.Solve(obj, prec, maxIter)
    peak_field = abs(rad.Fld(obj, 'bz', pf_loc)) #peak field / T
    k = sc.e * peak_field * per * 1e-3 / (2 * sc.pi * sc.m_e * sc.c)
    if lprint:
        print("peak field:", peak_field, "(calculated at given location",pf_loc,")\nperiod is", per, "(given input)\nk is", k)

    return k

def undulatorK(obj, xyz_start, xyz_end, n = 1000, prec = 1e-5, maxIter = 10000, lsove = True, lprint = True):
    """
    compute undulator K value
    arguments:
      obj = undulator object
      xyz_start = start point [x, y, z] in field calculation
      xyz_end = end point [x, y, z] in field calculation
      n = number of points in field calculation
      prec = precision goal for this computation
      maxIter = maximum allowed iterations
      lsove: if true, run rad.Solve
      lprint: if true, print results
    return:
      K = (e B_0 \lambda_u) / (2\pi m_e c)
    """
    # --- Sampling
    x0, y0, z0 = xyz_start
    x1, y1, z1 = xyz_end
    # Steps
    dx = (x1 - x0) / (n - 1)
    dy = (y1 - y0) / (n - 1)
    dz = (z1 - z0) / (n - 1)
    # Positions
    x = [x0 + k * dx for k in range(n)]
    y = [y0 + k * dy for k in range(n)]
    z = [z0 + k * dz for k in range(n)]
    xyz = [[x[k], y[k], z[k]] for k in range(n)]
    
    if lsove: 
        res = rad.Solve(obj, prec, maxIter)
        
    fields = rad.Fld(obj, 'bz', xyz)
    peak_field = max([abs(fieldb) for fieldb in fields]) #peak field / T
    fields_fft = np.fft.fft(fields)
    fields_fft_abs = [abs(fieldfft) for fieldfft in fields_fft]
    fft_per = 1/(fields_fft_abs.index(max(fields_fft_abs))/dy/n)
    k = sc.e * peak_field * fft_per * 1e-3 / (2 * sc.pi * sc.m_e * sc.c)
    
    if lprint:
        print("peak field:", peak_field, "(obtained by searching maximum value)\nperiod is",
              fft_per, "(obtained from FFT)\nk is", k,"(calcuated using maximum searched peak field)\n")
    return k

In [ ]:
undulatorK_simple(und,per)

In [ ]:
undulatorK(und, [0,-per*nper/2,0], [0,per*nper/2,0])

## _Generating an Undulator Map File_
An undulator (or wiggler)  must be created before executing any cell of this section. A suggestion is that: go to "Build and visualize an undulator" section and re-build the desired undulator. It is also a good strategy to check that the undulator has the desired geometry and that it is producing the correct field using the two previous sections before creating the tracking file.

In [ ]:
obj = und     #object (undulator), re-build the desired undulator to ensure the correct geometry inputs
p0 = [0,-per*nper/2,0]     # the starting point of longitudinal integration
vl = [0,1,0]     # longitudinal integration direction
vt = [1,0,0]     # one of the transverse direction (horizontal)
r1 = gap*0.75     #range of the transverse grid along vt (horizontal)
np1 = 21     #number of points in transverse direction vt (horizontal)
r2 = gap*0.75     #range of the transverse grid along (vt cross vl, verticle)
np2 = 21     #number of points in transverse direction (vt cross vl, verticle)
# default paras:
dpar = [1,8,0,0]    #[maximum number of magnetic field harmonics to treat:1,number of longitudinal points:8,steps of transverse differentiation:0,0]
unit = 'T2m2'     #the units for the resulting 2nd order kick values T2m2 or rad or microrad
en = 1     #eletron energy in GeV (required only if units are rad or microrad)
oFormat = 'fix'     #the format of the output data string: fix or tab

km = rad.FldFocKickPer(obj,p0,vl,per,nper,vt,r1,np1,r2,np2)

### _plot the kick maps_

In [ ]:
hKick = np.array(km[0])
vKick = np.array(km[1])
lmSqr = np.array(km[2])
xmesh,zmesh= np.meshgrid(np.array(km[3]),np.array(km[4]))

# horizontal kick
hkm = plt.axes(projection='3d')
hkm.plot_surface(xmesh, zmesh,hKick,  rstride=1, cstride=1,
                cmap='viridis', edgecolor='none')
hkm.set_title('horizontal kick');

In [ ]:
# verticle kick
vkm = plt.axes(projection='3d')
vkm.plot_surface(xmesh, zmesh, vKick,  rstride=1, cstride=1,
                cmap='viridis', edgecolor='none')
vkm.set_title('vertical kick');

In [ ]:
# longitudinally-integrated squared transverse magnetic field
lmsqr = plt.axes(projection='3d')
lmsqr.plot_surface(xmesh, zmesh, lmSqr,  rstride=1, cstride=1,
                cmap='viridis', edgecolor='none')
lmsqr.set_title('longitudinally-integrated squared transverse magnetic field');

### _save the map to text file_

In [ ]:
with open("km.txt", "w") as km_text:
    km_text.write(km[5])